In [2]:
import boto3
import mlflow

s3 = boto3.client('s3')
bucket = 'wildfires'
prefix = 'sagemaker/fire-image-classification'

# Create dummy files
dummy_content = "dummy file content"
s3.put_object(Bucket=bucket, Key=f'{prefix}/train/dummy.txt', Body=dummy_content)
s3.put_object(Bucket=bucket, Key=f'{prefix}/test/dummy.txt', Body=dummy_content)

# Experiments
tracking_server_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:mlflow-tracking-server/wildfire-mj'
experiment_name = 'wildfire-resnet-job'

In [4]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch

role = sagemaker.get_execution_role()
bucket = 'wildfires'
prefix = 'sagemaker/fire-image-classification'

# Upload the training script to S3
s3 = boto3.client('s3')
s3.upload_file('../Training/train.py', bucket, f'{prefix}/train.py')

estimator = PyTorch(
    entry_point='train.py',
    role=role,
    framework_version='1.9.0',
    py_version='py38',
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    hyperparameters={
        'num-epochs': 10,
        'batch-size': 32,
        'learning-rate': 0.1
    },
    source_dir=f's3://{bucket}/{prefix}/'
)

# Define input data channels
train_data = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/train', content_type='application/x-image')
test_data = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/test', content_type='application/x-image')

# Fit the estimator
estimator.fit({'train': train_data, 'test': test_data})
